In [82]:
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import MessagesPlaceholder
from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), temperature=0.7, model='gpt-3.5-turbo-0125')

In [2]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="ls__95354472777f469dbda1536504c248fb"
LANGCHAIN_PROJECT="cogito"

In [3]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [4]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [5]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")
    c: int = Field(description="third number")

In [6]:
class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "if there are only 2 numbers passed dont use it"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = False
    
    

    def _run(
        self, a: int, b: int, c: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b*c

In [7]:
tool = CustomCalculatorTool(is_single_input=True)

In [8]:
tool.description

'if there are only 2 numbers passed dont use it'

In [9]:
tools = [tool]

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate


In [11]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

In [53]:
class Type1(BaseModel):
    type: str = Field(description="Either chat or goals")
    message: str = Field(description="leave users message untouched here")



# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Type1)

prompt = ChatPromptTemplate.from_messages( [
    (
            "system",
            "You are an AI mentor that helps people stay motivated, so you store their goals. You need to decide whether the message you got from the user is about their specific goals, may be they want to add a new goal, or to extract an exisiting one, or whether they just want to chat. In the first case return just word goals. In other case return word chat. \n{format_instructions} \n The users message is following: {input} "
        ), 
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
]
)

In [54]:
firstChain = LLMChain(llm=llm, prompt=prompt)

In [62]:
chain11 = prompt | llm | parser

In [55]:
chaaain = firstChain

In [63]:
output = chain11.invoke({"input": "i want to watch 1 leacture a day not 2", "format_instructions":parser.get_format_instructions() })

In [64]:
output

Type1(type='goals', message='i want to watch 1 leacture a day not 2')

In [60]:
output = output.get("text")

In [61]:
obj = parser.parse(output)

In [59]:
obj.type

'goals'

In [90]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI mentor that helps people stay motivated, so you store their goals. You need to decide whether the message you got from the user is about their specific goals, may be they want to add a new goal, or to extract an exisiting one, or whether they just want to chat. "
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        
    ]
)

In [75]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [76]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [67]:
JustChatPrompt = ChatPromptTemplate.from_messages( [
    (
            "system",
            "You are an AI mentor that helps people stay motivated. Answer to the user according to this description. The users message is following: {input} "
        ), 
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
])

In [68]:
ChainJustChat = JustChatPrompt | llm

In [70]:
output = await ChainJustChat.ainvoke({"input": "hello how r u doing?"})

In [73]:
output.content

"Hello! I'm here and ready to help you stay motivated. How can I support you today?"

In [97]:
class AddGoalData(BaseModel):
    goal_name: str = Field(description="it is the name of the goal")
    goal_description: str = Field(description="Full description of a goal. Everything that you got from the user.")
    goal_end_state: str = Field(description="What should be done to consider a goal achieved.")
    user_id: int = Field(description="User's id passed by with the context")

class AddGoalTool(BaseTool):
    name = "Add users new goal to DB"
    description = "If a user has provided all the required info to make a record about his new goal, then use this tool "
    args_schema: Type[BaseModel] = AddGoalData

    def _run(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:

     
        return "Just run"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Add goal to a DB. """
        pass
        # If adding was succesfull return "Goal added sucessfully", else return "Mistake, try adding goal again!"
        return "Goal added sucessfully"

In [98]:
# Prompt for adding goal agent
AddingGoalAgentPrompt = ChatPromptTemplate.from_messages( [
    (
            "system",
            "You are an AI mentor that helps people stay motivated. User wants to add a new goal, therefore check a user"+
            "has provided all the required info. Required info: description of the goal, end of goal (under what circumstances should the goal be considered finished and achieved). The id of the user if: {user_id}. Consider also chat history: {chat_history}. The users message is following: {input} "
        ), 
        ("placeholder", "{chat_history}"),
        MessagesPlaceholder("agent_scratchpad"),
        ("human", "{input}"),
]
)

In [99]:
tool = AddGoalTool()
agent = create_tool_calling_agent(llm, tools= [tool ], prompt=AddingGoalAgentPrompt)
agent_executor = AgentExecutor(agent=agent, tools= [tool], verbose=True)

In [102]:
output = agent_executor.ainvoke({"input":"hello" ,"user_id": 505505})

In [103]:
output

<coroutine object Chain.ainvoke at 0x1507c4340>

In [ ]:
output.content

AttributeError: 'coroutine' object has no attribute 'content'

In [169]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = llm


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="Either chat or goals")

    # You can add custom validation logic easily with Pydantic.
    


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="You are an AI mentor that helps people stay motivated, so you store their goals. You need to decide whether the message you got from the user is about their specific goals, may be they want to add a new goal, or to extract an exisiting one, or whether they just want to chat.\n{format_instructions}\n The users message is following: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Lets talk about weather."})
parser.invoke(output)

Joke(setup='chat')

In [170]:
type(output)

langchain_core.messages.ai.AIMessage